In [ ]:
from __future__ import unicode_literals

import sys, os
BIN = os.path.expanduser("../../")
sys.path.append(BIN)

import numpy as np
from scipy.constants import m_p, c, e
import matplotlib.pyplot as plt
%matplotlib inline

import copy
import itertools

from test_tools import generate_objects, BunchTracker, track, compare_traces, compare_projections

from PyHEADTAIL_feedback.feedback import IdealBunchFeedback,IdealSliceFeedback,OneboxFeedback
from PyHEADTAIL_feedback.processors import ChargeWeighter, Averager, Bypass

np.random.seed(0)

In [ ]:
""" 
    This example shows howto use ideal feedback systems in PyHEADTAIL simulations. In order to avoid writing
    unnecessary code here, the used bunch, slicer and transfer maps are created with a function written to
    file test_tools.py in this folder.
"""

In [ ]:
# basic parameters of the simulation
n_macroparticles = 10000
n_slices = 20
n_segments = 5
n_sigma_z = 3 

n_turns = 150

bunch_ref, slicer_ref,trans_map, long_map = generate_objects(n_macroparticles,n_segments, n_slices,n_sigma_z)

In [ ]:
# this creates an artificially kicked bunch, which will be damped with different feedback systems

slice_set = bunch_ref.get_slices(slicer_ref, statistics=True)
p_idx = slice_set.particles_within_cuts
s_idx = slice_set.slice_index_of_particle.take(p_idx)

#random kicks
kick_x = 0.003*(-1.0+2*np.random.rand(n_slices))
kick_y = 0.003*(-1.0+2*np.random.rand(n_slices))

for p_id, s_id in itertools.izip(p_idx,s_idx):
    bunch_ref.x[p_id] += kick_x[s_id]
    bunch_ref.y[p_id] += kick_y[s_id]
    
bunch_init = copy.deepcopy(bunch_ref)
tracker_ref = BunchTracker(bunch_init)
maps_ref = [i for i in trans_map] + [long_map]
track(n_turns, bunch_init,maps_ref,tracker_ref)

In [ ]:
# In the used code, the parameter 'gain' is determined as a fraction of the input signal if the used signal 
# processors would bypass the signalperfectly

# There are two ways to set a gain value. If only one value is given, same value will be used both
# in horizontal and vertical plane. If two values are given, separed values (x,y) are used for horizontal and
# vertical planes

# feedback_gain = 0.1
feedback_gain = (0.1,0.4)

In [ ]:
# The simplest possible feedback is a bunch feedback. It takes the mean_xp and mean_yp values of the bunch 
# and correct those values a fraction of gain.

# The bunch is created by copying the bunch created in earlier
bunch_IdealBunchFB = copy.deepcopy(bunch_ref)

# A tracker object from test_tools.py is used for recording the values of the bunch during the bunch traking
tracker_IdealBunchFB = BunchTracker(bunch_IdealBunchFB)

# A PyHEADTAIL map element for the total map is created by creating IdealBunchFeedback() object. The only
# input parameter is the gain
feedback_map = IdealBunchFeedback(feedback_gain)
total_map_FB = [i for i in trans_map] + [long_map] + [feedback_map]

# 
track(n_turns, bunch_IdealBunchFB,total_map_FB,tracker_IdealBunchFB)

In [ ]:
# The most ideal feedback is slice feedback. It takes the mean_xp and mean_yp values for each slice and 
# correct them a fraction of gain.

bunch_IdealSliceFB = copy.deepcopy(bunch_ref)
tracker_IdealSliceFB = BunchTracker(bunch_IdealSliceFB)
slicer_IdealSliceFB = copy.deepcopy(slicer_ref)

# A PyHEADTAIL map element for the total map is created by creating IdealBunchFeedback() object. The only
# input parameter is the gain and a slicer for the bunch
feedback_map = IdealSliceFeedback(feedback_gain,slicer_IdealSliceFB)
total_map_FB = [i for i in trans_map] + [long_map] + [feedback_map]

# The damping of the artificially kicked bunch can be observed, when 
# This is done by using the track() function from 

track(n_turns, bunch_IdealSliceFB,total_map_FB,tracker_IdealSliceFB)

In [ ]:
# The examples above utilize separately programmed classes for ideal feedbacks. The actual feedback module
# utilizes objects namely signal processors for modifying signal between pickup and kicker. In order to learn
# this concept, we build the above introduced ideal feedbacks by using OneboxFeedback object.
#
# The easiest to implement is ideal slice feedback, which has been presented at first.

bunch_OneBox_slice = copy.deepcopy(bunch_ref)
tracker_OneBox_slice = BunchTracker(bunch_OneBox_slice)
slicer_OneBox_slice = copy.deepcopy(slicer_ref)


# By the defaul, Onebox feedback takes mean xp/yp values of the slices and gives that as an input signal for 
# signal processors. The signal goes throught all the signal processors, which are given as lists for
# horizontal and vertical planes. The final correction for mean xp/yp values of the slices is the signal from
# the signal processors, which is multiplied with a gain value. The definition for the gain value is that it 
# is a fraction of the mean xp/yp values of the slices corrected if the signal is not modified in signal 
# processors.
#
# In the ideal slice feedback, signals form the slices are not modified. Thus, a processor Bypass() is used
# in both vertical and horizontal planes, which bypasses the singnal without any modification.

processors_slice_x = [Bypass()]
processors_slice_y = [Bypass()]

# A PyHEADTAIL map element is created by calling OneboxFeedback(...) object. OneboxFeedback requires a gain
# value, a PyHEADTAIL slicer object and lists of signal processors used in horizontal and vertical plane
# as input arguments
feedback_map = OneboxFeedback(feedback_gain,slicer_OneBox_slice,processors_slice_x,processors_slice_y)
total_map_OneBox_slice = [i for i in trans_map] + [long_map] + [feedback_map]

track(n_turns, bunch_OneBox_slice,total_map_OneBox_slice,tracker_OneBox_slice)

In [ ]:
bunch_OneBox_bunch = copy.deepcopy(bunch_ref)
tracker_OneBox_bunch = BunchTracker(bunch_OneBox_bunch)
slicer_OneBox_bunch = copy.deepcopy(slicer_ref)

# The ideal bunch feedback makes corrections to mean xp/yp values of the bunch. In order to implement this 
# the input signal for the signal processors (mean xp/yp values of the slices) must be modified to the mean 
# xp/yp value of the bunch with signal processors. Because there are different number particles in each 
# slice this can not be done by simply averaging values of the slices, but the signal must be weighted by 
# a number of macro particles per slice at first. The weighting can be done by using ChargeWeighter(...). 
# By the default, the maximum weight in ChargeWeighter is normalized to be one, but, in this case, an average 
# weight must be 1, so the input parameter for the normalization is used. 

processors_bunch_x = [
    ChargeWeighter(normalization = 'average_weight'),
    Averager()
]
processors_bunch_y = [
    ChargeWeighter(normalization = 'average_weight'),
    Averager()
]

feedback_map = OneboxFeedback(feedback_gain,slicer_OneBox_bunch,processors_bunch_x,processors_bunch_y)
total_map_OneBox_bunch = [i for i in trans_map] + [long_map] + [feedback_map]

track(n_turns, bunch_OneBox_bunch,total_map_OneBox_bunch,tracker_OneBox_bunch)

In [ ]:
# In here, the traces from ideal slice and bunch feedbacks are presented as well as particle projections of 
# of the bunches after the tracking

compare_traces([tracker_ref,tracker_IdealSliceFB,tracker_IdealBunchFB],
               ['Reference, no FB', 'Ideal slice', 'Ideal bunch'])
compare_projections([bunch_init, bunch_IdealSliceFB, bunch_IdealBunchFB], 
                    ['Reference, no FB', 'Ideal slice FB', 'Ideal bunch FB'])

In [ ]:
# In here, the traces and bunch projection from ideal slice and bunch feedbacks are compared to those 
# implemented with OneboxFeedback

compare_traces([tracker_IdealSliceFB,tracker_IdealBunchFB, tracker_OneBox_slice, tracker_OneBox_bunch],
               ['Ideal slice', 'Ideal bunch','OneBox slice', 'OneBox bunch'])
compare_projections([bunch_IdealSliceFB, bunch_IdealBunchFB,bunch_OneBox_slice,bunch_OneBox_bunch], 
                    ['Ideal slice FB', 'Ideal bunch FB','OneBox slice', 'OneBox bunch'])